# F4

In [1]:
def reduction(p,F):
    div_occured = True
    while (p != 0 and div_occured):
        div_occured = False
        lt_p =  p.lt()
        for f in F:
            lt_f = f.lt()
            if (lt_p % lt_f == 0):
                p -= (lt_p//lt_f)*f
                div_occured = True
                break
    return p

def reduction_totale(p,F):
    p0 = 0
    while p != 0:
        p = reduction(p,F)
        lt_p = p.lt()
        p0 += lt_p
        p -= lt_p
    return p0

def minGBasis(F):
    if F == []:
        return []
    G = F.copy()
    G.sort(reverse = True)
    p = G[0]
    G.remove(p)
    G = minGBasis(G)
    if not(any([p.lt() % f.lt() == 0 for f in G])):
        G.append(p//p.lc())
    return G

def base_reduite(F):
    G = minGBasis(F)
    Gres = []
    for gi in G:
        Gtemp = G.copy()
        Gtemp.remove(gi)
        Gres.append(reduction_totale(gi,Gtemp))
    return Gres

In [2]:
def Pair(f,g):
    lm_f, lm_g = f.lm(), g.lm()
    ppcm = lcm(lm_f,lm_g)
    return (ppcm, ppcm//lm_f, f, ppcm//lm_g, g)

def SymbolicPreprocessing(L,G):
    F = set(L)
    Done = set([f.lm() for f in F])
    TF = T(F)
    while (TF != Done):
        m = (TF.difference(Done)).pop()
        Done = Done.union([m])
        for g in G:
            if (m % g.lm() == 0):
                F = F.union([(m//g.lm())*g])
    return F

def Reduction(L,G):
    F = SymbolicPreprocessing(L,G)
    ltF = [f.lm() for f in F]
    ls = list(T(F))
    ls.sort(reverse = True)
    F_tilde = []
    for f in F:
        f_lin = []
        for i in ls:
            if (i in f.monomials()):
                f_lin += [f.monomial_coefficient(i)]
            else:
                f_lin += [0]
        F_tilde += [f_lin]
    MF = matrix(F_tilde)
    MF.echelonize()
    f =  [sum([MF[j][i]*ls[i] for i in range(len(ls))]) for j in range(MF.dimensions()[0])]
    f = [fi for fi in f if (fi.lm() not in ltF)]
    return set(f)
        
def T(F):
    TF = []
    for fi in F:
        TF += fi.monomials()
    return set(TF)
    
def Sel(P):
    def min_pair(p):
        return p[0].degree()
    d = min_pair(min(P,key = min_pair))
    Pd = set([p for p in P if (p[0].degree() == d)])
    return Pd


In [3]:
def F4(F):
    G = set(F)
    P = set([Pair(i,j)for i,j in Subsets(G,2)])
    while P != set():
        Pd = Sel(list(P))
        P = P.difference(Pd)
        Left_Pd = set([p[1]*p[2] for p in Pd])
        Right_Pd = set([p[3]*p[4] for p in Pd])
        Ld = Left_Pd.union(Right_Pd)
        Fd_tilde = Reduction(Ld,G)
        LtG = [f.lm() for f in G]
        for h in Fd_tilde:
            if h != 0 and all([h.lm()%f != 0 for f in LtG]):
                P = P.union([Pair(h,g) for g in G])
                G = G.union([h])
    return base_reduite(list(G))

## Example

In [4]:
Q = PolynomialRing(QQ,4, names = 'abcd')
a,b,c,d = Q('a'),Q('b'),Q('c'),Q('d')
f1 = a*b*c*d-1
f2 = a*b*c+a*b*d+a*c*d+b*c*d
f3 = a*b+b*c+a*d+c*d
f4 = a+b+c+d
F = [f1,f2,f3,f4]


In [5]:
Q = PolynomialRing(QQ,3, names = 'xyz')
x,y,z = Q('x'),Q('y'),Q('z')
f1 = x^2+x*y-1
f2 = x^2-z^2
f3 = x*y+1
F = [f1,f2,f3]

In [6]:
IF = Ideal(F)

In [10]:
IF.groebner_basis()

[y^2 - 1/2, z^2 - 2, x + 2*y]

In [9]:
F4(F)

[x + 2*y, z^2 - 2, y^2 - 1/2]